# 1. 모델 구현

* https://www.tensorflow.org/guide/keras/sequential_model

```
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)
```

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
# padding='same' : 자동으로 패딩을 삽입해 입력값과 출력값의 크기를 맞춰줌
# padding='valid' : 패딩을 적용하지 않고 필터를 적용해서 출력값의 크기가 작아짐

def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            layers.Input(input_shape),



            # 1st
            layers.Conv2D(64, 3, strides=1, activation='relu',padding='same'), #CNN모델의 레이어, 64개의 필터, 3by3짜리 필터크기, 
            layers.Conv2D(64, 3, strides=1, activation='relu',padding='same'),
            # 두번 가중치를 주면 좀 더 곡선이 살아남
            layers.MaxPool2D(), # 연산량을 줄여주면서 그 안에 있는 특징을 추출하기위해 maxpool2d사용
            layers.BatchNormalization(), # 값들을 정규화.
            layers.Dropout(0.5), # 과대적합하지않게 # 50%를 학습시킴



            # 2nd
            layers.Conv2D(128, 3, strides=1, activation='relu',padding='same'), #CNN모델의 레이어, 128개의 필터, 3by3짜리 필터크기, 
            layers.Conv2D(128, 3, strides=1, activation='relu',padding='same'),
            # 두번 가중치를 주면 좀 더 곡선이 살아남
            layers.MaxPool2D(), # 연산량을 줄여주면서 그 안에 있는 특징을 추출하기위해 maxpool2d사용
            layers.BatchNormalization(), # 값들을 정규화.
            layers.Dropout(0.3), # 과대적합하지않게 # 30%를 학습시킴        



            # 레이어를 두번 사용


            # FC
            layers.GlobalMaxPool2D(), # 여태까지 왔던 모든 데이터를 maxpool함
            layers.Dense(128, activation="relu"),
            layers.Dense(1, activation='sigmoid'), # 시그모이드로 판단
        ]
    )
    return model

In [11]:
input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 128, 128, 64)      0         
                                                                 
 conv2d_14 (Conv2D)          (None, 128, 128, 128)    

In [14]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        
        
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu',padding='same'), #CNN모델의 레이어, 64개의 필터, 3by3짜리 필터크기, 
                layers.Conv2D(64, 3, strides=1, activation='relu',padding='same'),
                # 두번 가중치를 주면 좀 더 곡선이 살아남
                layers.MaxPool2D(), # 연산량을 줄여주면서 그 안에 있는 특징을 추출하기위해 maxpool2d사용
                layers.BatchNormalization(), # 값들을 정규화.
                layers.Dropout(0.5), # 과대적합하지않게 # 50%를 학습시킴           
            ], name = 'conv_block_1' # 뒤에 이름 붙일 수 있음
        )
        
        
        
        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu',padding='same'), #CNN모델의 레이어, 128개의 필터, 3by3짜리 필터크기, 
                layers.Conv2D(128, 3, strides=1, activation='relu',padding='same'),
                # 두번 가중치를 주면 좀 더 곡선이 살아남
                layers.MaxPool2D(), # 연산량을 줄여주면서 그 안에 있는 특징을 추출하기위해 maxpool2d사용
                layers.BatchNormalization(), # 값들을 정규화.
                layers.Dropout(0.3), # 과대적합하지않게 # 30%를 학습시킴                   
            ], name = 'conv_block_2' # 뒤에 이름 붙일 수 있음
        )
        
        
        
        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(), # 여태까지 왔던 모든 데이터를 maxpool함
                layers.Dense(128, activation="relu"),
                layers.Dense(1, activation='sigmoid'), # 시그모이드로 판단                
            ], name = 'classifier'
        )

        
        
        
    def call(self, input_tensor, training=False): # 객체를 생성하면 자동으로 호출되는 메소드
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        
        return x

In [15]:
input_shape = (None, 256, 256, 3) # 배치들어가는거라서 안넣음, 채널3짜리 
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block_1 (Sequential)   (None, 128, 128, 64)      38976     
                                                                 
 conv_block_2 (Sequential)   (None, 64, 64, 128)       221952    
                                                                 
 classifier (Sequential)     (None, 1)                 16641     
                                                                 
Total params: 277,569
Trainable params: 277,185
Non-trainable params: 384
_________________________________________________________________


In [19]:
model.compile( # compile함수 사용해서 설정값 설정
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics='accuracy'
)